In [13]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 8.1 MB/s eta 0:00:00


In [12]:
!pip install datasets

ERROR: Invalid requirement: 'datasets,': Expected end or semicolon (after name and no valid version specifier)
    datasets,
            ^


# I. Import Required Library

In [55]:
from datasets import load_dataset
import numpy as np
import pandas as pd
import re
import emoji
import unicodedata
from google.colab import files

In [4]:
ds = load_dataset("htdung167/ViHSD")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


train.csv:   0%|          | 0.00/1.62M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/177k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/442k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/24048 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2672 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6680 [00:00<?, ? examples/s]

In [6]:
df_train = pd.DataFrame(ds['train'])
df_valid = pd.DataFrame(ds['validation'])
df_test = pd.DataFrame(ds['test'])

In [7]:
print(ds.shape)

{'train': (24048, 2), 'validation': (2672, 2), 'test': (6680, 2)}


In [8]:
print(df_train.columns)

Index(['free_text', 'label_id'], dtype='object')


In [44]:
print(df_train.isnull().sum())
print(df_valid.isnull().sum())
print(df_test.isnull().sum())

free_text       0
label_id        0
binary_label    0
dtype: int64
free_text       0
label_id        0
binary_label    0
dtype: int64
free_text       0
label_id        0
binary_label    0
dtype: int64


# II. Convert to binary label (toxic || non-toxic)

In [24]:
print(df_train)

                                               free_text  label_id  \
0      Em được làm fan cứng luôn rồi nè ❤️ reaction q...         0   
1      Đúng là bọn mắt híp lò xo thụt :))) bên việt n...         2   
2               Đậu Văn Cường giờ giống thằng sida hơn à         0   
3      CÔN ĐỒ CỤC SÚC VÔ NHÂN TÍNH ĐỀ NGHI VN. NHÀ NƯ...         2   
4      Từ lý thuyết đến thực hành là cả 1 câu chuyện ...         0   
...                                                  ...       ...   
24043                  Thả tim cho sự cống hiến của ngài         0   
24044                                             Thua !         0   
24045  Ở Việt Nam thì có khánh sky..hải ngoại có trần...         0   
24046    1:22 ai lớp du chu cờ moo, Duyên dáng quá cô ơi         0   
24047  Cụ này mắt mới cắt mý sao ta kkk đấy vnch đấy ...         0   

       binary_label  
0                 0  
1                 1  
2                 0  
3                 1  
4                 0  
...             ...  
24043

In [23]:
binary_mapping = {0: 0, 1: 1, 2: 1}

df_train['binary_label'] = df_train['label_id'].map(binary_mapping)
df_valid['binary_label'] = df_valid['label_id'].map(binary_mapping)
df_test['binary_label'] = df_test['label_id'].map(binary_mapping)

In [32]:
df_train.dtypes

,0
free_text,object
label_id,int64
binary_label,int64


# III. Convert emoji & special symbols to text

In [51]:
def convert_special_symbols(text):
    text = re.sub(r":\)+", " cười lớn ", text)  # Cười lớn
    text = re.sub(r":\(+", " buồn ", text)      # Buồn
    text = re.sub(r":v", " cười mỉa mai ", text) # Cười mỉa mai
    text = re.sub(r"=]]+", " cười sảng khoái ", text) # Cười sảng khoái
    text = re.sub(r":3", " dễ thương ", text) # Dễ thương
    text = re.sub(r"=\)\)+", " cười lăn lộn ", text) # Cười lăn lộn

    # Biểu tượng cảm xúc
    text = re.sub(r"<3", " trái tim ", text)
    text = re.sub(r"</3", " tan vỡ ", text)
    text = re.sub(r"~~+", " châm biếm ", text)
    text = re.sub(r"\^\^", " vui vẻ ", text)
    text = re.sub(r"T_T|;-;", " khóc ", text)

    return text

def clean_text(text):
    # Chuyển sang string nếu không phải là string
    if not isinstance(text, str):
        text = str(text)

    # Bước 1: Chuẩn hóa Unicode
    text = unicodedata.normalize("NFC", text)


    # Bước 2: Xử lý các ký hiệu cảm xúc
    text = convert_special_symbols(text)

    # Bước 3: Xóa ký tự đặc biệt nhưng giữ lại dấu tiếng Việt
    text = re.sub(r"[^\w\s\u00C0-\u1FFF\u2C00-\uD7FF]", " ", text)

    # Bước 4: Xóa khoảng trắng thừa giữa các từ
    text = re.sub(r"\s+", " ", text)


    # Bước 5: Loại bỏ khoảng trắng thừa
    return text.lower().strip()

In [52]:
df_train['free_text'] = df_train['free_text'].apply(clean_text)
df_valid['free_text'] = df_valid['free_text'].apply(clean_text)
df_test['free_text'] = df_test['free_text'].apply(clean_text)

In [53]:
df_train

,free_text,label_id,binary_label
0,em được làm fan cứng luôn rồi nè reaction quá ...,0,0
1,đúng là bọn mắt híp lò xo thụt cười lớn bên vi...,2,1
2,đậu văn cường giờ giống thằng sida hơn à,0,0
3,côn đồ cục súc vô nhân tính đề nghi vn nhà nướ...,2,1
4,từ lý thuyết đến thực hành là cả 1 câu chuyện ...,0,0
...,...,...,...
23842,thả tim cho sự cống hiến của ngài,0,0
23843,thua,0,0
23844,ở việt nam thì có khánh sky hải ngoại có trần ...,0,0
23845,1 22 ai lớp du chu cờ moo duyên dáng quá cô ơi,0,0


In [54]:
# Xoá các hàng đã bị mất dữ liệu khi xử lý clean text bên trên
df_train = df_train[df_train['free_text'].str.strip() != ""].reset_index(drop=True)
df_valid = df_valid[df_valid['free_text'].str.strip() != ""].reset_index(drop=True)
df_test = df_test[df_test['free_text'].str.strip() != ""].reset_index(drop=True)

In [56]:

# Lưu file CSV
df_train.to_csv("preprocessed_train.csv", index=False)
df_valid.to_csv("preprocessed_valid.csv", index=False)
df_test.to_csv("preprocessed_test.csv", index=False)

# Tải xuống về máy
files.download("preprocessed_train.csv")
files.download("preprocessed_valid.csv")d
files.download("preprocessed_test.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>